# Lesson 3: Fundamental Concepts of Agentic AI
## Practical Examples and Exercises

This notebook contains practical examples and exercises to reinforce the concepts covered in Lesson 3.

**Learning Objectives:**
- Understand the difference between chatbots and autonomous agents
- Implement basic agentic patterns
- Design and use tools in an agentic system
- Practice building simple agents

## Setup and Installation

First, let's install and import required libraries.

In [ ]:
# Install required packages
!pip install -q groq google-generativeai python-dotenv

In [ ]:
import os
import json
from datetime import datetime
from typing import List, Dict, Any
import groq
import google.generativeai as genai

## API Configuration

Configure your API keys for Groq and Google Gemini.

**Get your API keys:**
- Groq: https://console.groq.com
- Gemini: https://aistudio.google.com/app/apikey

In [ ]:
# Set your API keys here or use environment variables
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "your-groq-api-key-here")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "your-gemini-api-key-here")

# Initialize clients
groq_client = groq.Groq(api_key=GROQ_API_KEY)
genai.configure(api_key=GEMINI_API_KEY)

---

# Part 1: Theory Recap

## Chatbot vs Agent

**Traditional Chatbot:**
- Reactive: responds to user input
- No planning capability
- Cannot use external tools
- Limited memory

**Autonomous Agent:**
- Proactive: pursues goals
- Plans multi-step actions
- Uses external tools (APIs, databases, calculators)
- Strategic memory management

## Agent Execution Cycle

```
1. PERCEIVE → Receive input
2. REASON → Analyze and plan
3. ACT → Execute action (use tool)
4. OBSERVE → Get result
5. EVALUATE → Check if goal reached
   ↓ (if not done)
   Back to REASON
```

## ReAct Pattern

**Re**asoning + **Act**ing
```
Thought: "I need to get campaign data"
Action: call_api("get_campaign", params)
Observation: {"reach": "45%"}
Thought: "Now I can answer"
Final Answer: "The reach is 45%"
```

---

# Part 2: Example 1 - Simulating ReAct Pattern

Let's manually implement a simple ReAct cycle to understand how agents think.

In [ ]:
class SimpleReActAgent:
    """
    A simple implementation of ReAct pattern for educational purposes.
    This simulates how an agent alternates between reasoning and acting.
    """
    
    def __init__(self):
        self.memory = []
        self.max_iterations = 5
        
    def perceive(self, user_input: str):
        """Receive input from user"""
        print(f"\n[PERCEIVE] User says: {user_input}")
        return user_input
    
    def reason(self, context: str, iteration: int) -> Dict[str, Any]:
        """Decide what action to take"""
        print(f"\n[REASON - Iteration {iteration}] Thinking about what to do...")
        
        # Simulate reasoning (in real agent, this would be LLM call)
        if "campaign" in context.lower() and "reach" in context.lower():
            thought = "User wants reach data for a campaign. I need to call the API."
            action = "get_campaign_data"
            params = {"campaign_id": "autumn_2024", "metric": "reach"}
        elif "compare" in context.lower():
            thought = "User wants to compare campaigns. I need to get data for both."
            action = "compare_campaigns"
            params = {"campaigns": ["autumn_2024", "winter_2024"]}
        else:
            thought = "I have enough information to answer."
            action = "final_answer"
            params = {}
            
        print(f"[REASON] Thought: {thought}")
        return {"thought": thought, "action": action, "params": params}
    
    def act(self, action: str, params: Dict) -> Any:
        """Execute the decided action"""
        print(f"\n[ACT] Executing: {action} with params: {params}")
        
        # Simulate tool execution (in real agent, this would call actual tools)
        if action == "get_campaign_data":
            # Simulate API response
            result = {"reach": "48%", "frequency": 3.8, "impressions": 5200000}
        elif action == "compare_campaigns":
            result = {
                "autumn_2024": {"reach": "48%"},
                "winter_2024": {"reach": "52%"}
            }
        else:
            result = None
            
        return result
    
    def observe(self, result: Any) -> str:
        """Process the result of the action"""
        print(f"\n[OBSERVE] Result: {result}")
        observation = f"Tool returned: {json.dumps(result, indent=2)}"
        self.memory.append(observation)
        return observation
    
    def evaluate(self, action: str) -> bool:
        """Check if we have reached the goal"""
        print(f"\n[EVALUATE] Checking if goal is reached...")
        goal_reached = action == "final_answer"
        print(f"[EVALUATE] Goal reached: {goal_reached}")
        return goal_reached
    
    def run(self, user_input: str):
        """Execute the full ReAct cycle"""
        print("="*60)
        print("STARTING REACT AGENT CYCLE")
        print("="*60)
        
        context = self.perceive(user_input)
        
        for iteration in range(1, self.max_iterations + 1):
            # Reason
            decision = self.reason(context, iteration)
            
            # Act
            result = self.act(decision["action"], decision["params"])
            
            # Evaluate
            if self.evaluate(decision["action"]):
                print("\n" + "="*60)
                print("AGENT COMPLETED TASK")
                print("="*60)
                break
                
            # Observe
            observation = self.observe(result)
            context = context + "\n" + observation
            
        return self.memory

In [ ]:
# Test the simple ReAct agent
agent = SimpleReActAgent()
agent.run("What is the reach of the autumn campaign?")

---

# Part 3: Example 2 - Tool Definition

Let's define tools that an agent could use to interact with the TTVAM API.

In [ ]:
# Define tool schemas (how we tell the agent what tools are available)

TTVAM_TOOLS = [
    {
        "name": "get_campaign_performance",
        "description": """Retrieves performance data for an advertising campaign 
                          from TTVAM system. Use this when you need metrics like 
                          reach, frequency, or impressions for a specific campaign.""",
        "parameters": {
            "type": "object",
            "properties": {
                "spotgate_code": {
                    "type": "string",
                    "description": "The Spotgate code identifying the campaign"
                },
                "period_start": {
                    "type": "string",
                    "description": "Start date in YYYY-MM-DD format"
                },
                "period_end": {
                    "type": "string",
                    "description": "End date in YYYY-MM-DD format"
                },
                "target": {
                    "type": "object",
                    "description": "Demographic target (optional)"
                }
            },
            "required": ["spotgate_code", "period_start", "period_end"]
        }
    },
    {
        "name": "calculate_kpi",
        "description": """Calculates derived KPIs from campaign data. 
                          Use this after retrieving raw data to compute 
                          metrics like CPM, cost per reach point, etc.""",
        "parameters": {
            "type": "object",
            "properties": {
                "impressions": {
                    "type": "number",
                    "description": "Total impressions"
                },
                "reach": {
                    "type": "number",
                    "description": "Reach percentage"
                },
                "budget": {
                    "type": "number",
                    "description": "Campaign budget in currency"
                }
            },
            "required": ["impressions", "reach", "budget"]
        }
    },
    {
        "name": "compare_campaigns",
        "description": """Compares performance of multiple campaigns. 
                          Use this when user asks to compare or benchmark campaigns.""",
        "parameters": {
            "type": "object",
            "properties": {
                "campaign_ids": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of campaign Spotgate codes to compare"
                },
                "metrics": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "Metrics to compare (reach, frequency, impressions)"
                }
            },
            "required": ["campaign_ids", "metrics"]
        }
    }
]

# Display tool definitions
print("Available Tools for TTVAM Agent:\n")
for tool in TTVAM_TOOLS:
    print(f"Tool: {tool['name']}")
    print(f"Description: {tool['description']}")
    print(f"Parameters: {list(tool['parameters']['properties'].keys())}")
    print("-" * 60)

### Tool Implementation

Now let's implement the actual functions that execute these tools.

In [ ]:
def get_campaign_performance(spotgate_code: str, period_start: str, 
                            period_end: str, target: Dict = None) -> Dict:
    """
    Simulates retrieving campaign performance data from TTVAM API.
    In production, this would make actual API calls.
    """
    print(f"\n[TOOL] Calling get_campaign_performance...")
    print(f"  Spotgate Code: {spotgate_code}")
    print(f"  Period: {period_start} to {period_end}")
    
    # Simulate API response
    mock_data = {
        "1234": {
            "reach": 48.5,
            "frequency": 3.8,
            "impressions": 5200000,
            "campaign_name": "Autumn Campaign 2024"
        },
        "5678": {
            "reach": 52.3,
            "frequency": 4.2,
            "impressions": 6100000,
            "campaign_name": "Winter Campaign 2024"
        }
    }
    
    result = mock_data.get(spotgate_code, {
        "reach": 45.0,
        "frequency": 3.5,
        "impressions": 4800000,
        "campaign_name": "Unknown Campaign"
    })
    
    print(f"[TOOL] Retrieved data: {result}")
    return result


def calculate_kpi(impressions: float, reach: float, budget: float) -> Dict:
    """
    Calculates derived KPIs from campaign metrics.
    """
    print(f"\n[TOOL] Calculating KPIs...")
    print(f"  Impressions: {impressions:,.0f}")
    print(f"  Reach: {reach}%")
    print(f"  Budget: €{budget:,.2f}")
    
    # Calculate KPIs
    cpm = (budget / impressions) * 1000
    cost_per_reach_point = budget / reach
    
    result = {
        "cpm": round(cpm, 2),
        "cost_per_reach_point": round(cost_per_reach_point, 2),
        "efficiency_score": round(reach / (cpm * 10), 2)
    }
    
    print(f"[TOOL] KPIs calculated: {result}")
    return result


def compare_campaigns(campaign_ids: List[str], metrics: List[str]) -> Dict:
    """
    Compares performance of multiple campaigns.
    """
    print(f"\n[TOOL] Comparing campaigns...")
    print(f"  Campaigns: {campaign_ids}")
    print(f"  Metrics: {metrics}")
    
    # Get data for each campaign
    comparison = {}
    for campaign_id in campaign_ids:
        data = get_campaign_performance(campaign_id, "2024-01-01", "2024-12-31")
        comparison[campaign_id] = {k: v for k, v in data.items() if k in metrics}
    
    print(f"[TOOL] Comparison result: {comparison}")
    return comparison


# Tool registry - maps tool names to functions
TOOL_REGISTRY = {
    "get_campaign_performance": get_campaign_performance,
    "calculate_kpi": calculate_kpi,
    "compare_campaigns": compare_campaigns
}

In [ ]:
# Test the tools
print("Testing TTVAM Tools:")
print("="*60)

# Test 1: Get campaign performance
result1 = get_campaign_performance("1234", "2024-03-01", "2024-03-31")

# Test 2: Calculate KPIs
result2 = calculate_kpi(5200000, 48.5, 50000)

# Test 3: Compare campaigns
result3 = compare_campaigns(["1234", "5678"], ["reach", "frequency", "impressions"])

---

# Part 4: Example 3 - Simple Agent with LLM

Now let's create a simple agent that uses an LLM (Groq or Gemini) to make decisions about which tools to use.

In [ ]:
def create_agent_prompt(user_query: str, available_tools: List[Dict], 
                       conversation_history: str = "") -> str:
    """
    Creates a prompt that instructs the LLM to act as an agent.
    """
    tools_description = "\n".join([
        f"- {tool['name']}: {tool['description']}"
        for tool in available_tools
    ])
    
    prompt = f"""You are an intelligent agent that helps analyze advertising campaigns.

You have access to the following tools:
{tools_description}

To use a tool, respond with a JSON object in this format:
{{
  "thought": "your reasoning about what to do",
  "action": "tool_name",
  "parameters": {{"param1": "value1", "param2": "value2"}}
}}

When you have all the information needed to answer the user's question, respond with:
{{
  "thought": "I have all the information needed",
  "action": "final_answer",
  "answer": "your complete answer to the user"
}}

Conversation history:
{conversation_history}

User query: {user_query}

Respond with JSON only, no other text:"""
    
    return prompt

In [ ]:
def run_simple_agent_groq(user_query: str, max_iterations: int = 5):
    """
    Runs a simple agent using Groq's Llama model.
    """
    print("="*60)
    print("RUNNING SIMPLE AGENT WITH GROQ")
    print("="*60)
    print(f"\nUser Query: {user_query}\n")
    
    conversation_history = ""
    
    for iteration in range(1, max_iterations + 1):
        print(f"\n--- Iteration {iteration} ---")
        
        # Create prompt
        prompt = create_agent_prompt(user_query, TTVAM_TOOLS, conversation_history)
        
        # Get LLM response
        try:
            response = groq_client.chat.completions.create(
                model="llama-3.1-70b-versatile",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1,
                max_tokens=500
            )
            
            agent_response = response.choices[0].message.content
            print(f"\n[AGENT RESPONSE]\n{agent_response}")
            
            # Parse JSON response
            try:
                decision = json.loads(agent_response)
            except json.JSONDecodeError:
                # Try to extract JSON from response
                import re
                json_match = re.search(r'\{.*\}', agent_response, re.DOTALL)
                if json_match:
                    decision = json.loads(json_match.group())
                else:
                    print("[ERROR] Could not parse agent response as JSON")
                    break
            
            print(f"\n[THOUGHT] {decision.get('thought', 'No thought provided')}")
            print(f"[ACTION] {decision.get('action', 'No action specified')}")
            
            # Check if agent wants to provide final answer
            if decision.get('action') == 'final_answer':
                print(f"\n{'='*60}")
                print("FINAL ANSWER:")
                print(f"{'='*60}")
                print(decision.get('answer', 'No answer provided'))
                break
            
            # Execute tool
            tool_name = decision.get('action')
            if tool_name in TOOL_REGISTRY:
                tool_function = TOOL_REGISTRY[tool_name]
                parameters = decision.get('parameters', {})
                
                result = tool_function(**parameters)
                
                # Add to conversation history
                conversation_history += f"\nAction taken: {tool_name}\n"
                conversation_history += f"Result: {json.dumps(result)}\n"
            else:
                print(f"[ERROR] Unknown tool: {tool_name}")
                break
                
        except Exception as e:
            print(f"[ERROR] {str(e)}")
            break
    
    print(f"\n{'='*60}")
    print("AGENT EXECUTION COMPLETED")
    print(f"{'='*60}")

In [ ]:
# Test the agent with a simple query
run_simple_agent_groq("What was the reach of campaign 1234?")

---

# Part 5: Exercises

Now it's your turn to practice!

## Exercise 1: Design a New Tool

Design a tool called `generate_report` that creates a summary report for a campaign.

**Requirements:**
- Tool name: `generate_report`
- Description: Should clearly explain what the tool does
- Parameters: 
  - `campaign_id` (string, required)
  - `report_type` (string, required): "executive" or "detailed"
  - `include_charts` (boolean, optional)

Complete the tool definition below:

In [ ]:
# Exercise 1: Your solution here

generate_report_tool = {
    "name": "generate_report",
    "description": """# TODO: Write a clear description of what this tool does
                      # Hint: Explain when an agent should use this tool
                      """,
    "parameters": {
        "type": "object",
        "properties": {
            # TODO: Define the parameters here
            # Use the TTVAM_TOOLS examples above as reference
        },
        "required": []  # TODO: List required parameters
    }
}

# Test your tool definition
print(json.dumps(generate_report_tool, indent=2))

## Exercise 2: Implement the Tool Function

Now implement the actual function for the `generate_report` tool.

The function should:
1. Call `get_campaign_performance` to get the data
2. Create a summary based on `report_type`
3. Return a formatted report as a dictionary

In [ ]:
# Exercise 2: Your solution here

def generate_report(campaign_id: str, report_type: str, include_charts: bool = False) -> Dict:
    """
    Generates a summary report for a campaign.
    
    Args:
        campaign_id: Spotgate code of the campaign
        report_type: Type of report ("executive" or "detailed")
        include_charts: Whether to include chart recommendations
    
    Returns:
        Dictionary containing the report
    """
    # TODO: Implement this function
    # Steps:
    # 1. Get campaign data using get_campaign_performance
    # 2. Create summary based on report_type
    # 3. If report_type == "executive", provide brief summary
    # 4. If report_type == "detailed", provide comprehensive summary
    # 5. If include_charts, suggest appropriate charts
    # 6. Return report as dictionary
    
    pass  # Remove this and add your implementation

# Test your function
# result = generate_report("1234", "executive", include_charts=True)
# print(json.dumps(result, indent=2))

## Exercise 3: Build an Agent Prompt

Create a prompt for an agent that specializes in comparing campaigns.

**Requirements:**
- The agent should be an expert in campaign comparison
- It should always use the `compare_campaigns` tool when asked to compare
- It should provide insights about which campaign performed better and why
- Include examples in your prompt (few-shot learning)

In [ ]:
# Exercise 3: Your solution here

def create_comparison_agent_prompt(user_query: str) -> str:
    """
    Creates a specialized prompt for a campaign comparison agent.
    """
    prompt = """# TODO: Create your agent prompt here
    
    Guidelines:
    1. Define the agent's role (campaign comparison expert)
    2. List available tools
    3. Provide examples of good comparisons (few-shot)
    4. Explain the expected output format
    5. Include the user query
    
    Example structure:
    - You are a...
    - You have access to...
    - Example 1: User asks "Compare X and Y" → You use tool...
    - Example 2: ...
    - Now answer: {user_query}
    """
    
    # TODO: Replace the above with your actual prompt
    
    return prompt

# Test your prompt
test_prompt = create_comparison_agent_prompt("Compare autumn and winter campaigns")
print(test_prompt)

## Exercise 4: Extend the Simple Agent

Modify the `run_simple_agent_groq` function to:
1. Add error handling for tool execution failures
2. Add a maximum token limit warning
3. Save the conversation history to a log file
4. Add support for the new `generate_report` tool

In [ ]:
# Exercise 4: Your solution here

def run_enhanced_agent(user_query: str, max_iterations: int = 5, log_file: str = "agent_log.txt"):
    """
    Enhanced version of the simple agent with additional features.
    """
    # TODO: Implement the enhanced agent
    # Add the following features:
    # 1. Try-except blocks for tool execution
    # 2. Warning if response is getting too long
    # 3. Write conversation to log file
    # 4. Support for generate_report tool
    
    pass  # Remove and implement

# Test your enhanced agent
# run_enhanced_agent("Generate an executive report for campaign 1234")

## Exercise 5: Analyze Agent Behavior

Run the simple agent with different queries and analyze its behavior.

**Queries to test:**
1. "What is the reach of campaign 1234?"
2. "Compare campaigns 1234 and 5678"
3. "Calculate the CPM for campaign 1234 with a budget of 50000 euros"
4. "Which campaign performed better: 1234 or 5678?"

For each query, note:
- How many iterations did the agent take?
- Which tools did it use?
- Was the final answer correct?
- Were there any errors or inefficiencies?

In [ ]:
# Exercise 5: Your analysis here

test_queries = [
    "What is the reach of campaign 1234?",
    "Compare campaigns 1234 and 5678",
    "Calculate the CPM for campaign 1234 with a budget of 50000 euros",
    "Which campaign performed better: 1234 or 5678?"
]

# TODO: Run each query and document your observations
# Create a summary table with your findings

for query in test_queries:
    print(f"\n\n{'='*80}")
    print(f"Testing query: {query}")
    print(f"{'='*80}")
    # Uncomment to run:
    # run_simple_agent_groq(query)
    
# TODO: Write your analysis below
analysis = """
Query 1 Analysis:
- Iterations: ?
- Tools used: ?
- Correct answer: Yes/No
- Observations: ...

Query 2 Analysis:
...
"""

print(analysis)

---

## Bonus Exercise: Multi-Step Planning

Create an agent that can handle a complex multi-step query:

**Query:** "Compare campaigns 1234 and 5678, calculate the CPM for both, and tell me which one is more cost-efficient."

This requires:
1. Getting data for both campaigns
2. Calculating KPIs for both
3. Comparing the results
4. Providing a recommendation

Implement this and observe how the agent plans its actions.

In [ ]:
# Bonus Exercise: Your solution here

complex_query = """Compare campaigns 1234 and 5678, calculate the CPM for both 
                   (assume budget of 50000 for 1234 and 60000 for 5678), 
                   and tell me which one is more cost-efficient."""

# TODO: Run the agent with this complex query
# Observe and document how it breaks down the task

# run_simple_agent_groq(complex_query, max_iterations=10)

---

## Conclusion

In this notebook, you learned:

1. **ReAct Pattern**: How agents alternate between reasoning and acting
2. **Tool Design**: How to define and implement tools for agents
3. **Agent Implementation**: How to create a simple agent using LLMs
4. **Practical Application**: How to apply these concepts to the TTVAM use case

**Next Steps:**
- Complete all exercises
- Experiment with different prompts
- Try adding new tools
- Test edge cases and error scenarios

**Preparation for Lesson 4:**
- Install Langchain: `pip install langchain langchain-groq langchain-google-genai`
- Review the Langchain documentation: https://python.langchain.com/docs/
- Think about how frameworks like Langchain can simplify agent development